# Lab: UAV-assisted wireless localization

_Fraida Fund_

* **Net ID**:
* **Name**:

You've been asked to contribute your machine learning expertise to a crucial and potentially life-saving mission.

A pair of hikers has gone missing in a national park, and they are believed to be critically injured. Fortunately, they have activated a wireless locator beacon, and it transmits a constant wireless signal from their location. Unfortunately, their beacon was not able to get a satellite fix, so their GPS position is not known.

To rescue the injured hikers, therefore, their location must be estimated using the signal strength of the wireless signal from the beacon: when a radio receiver is close to the beacon, the signal strength will be high. When a radio receiver is far from the beacon, the signal strength will be lower.  (The relationship is noisy, however; the wireless signal also fluctuates over time, even with a constant distance.)

You are going to fly an unmanned aerial vehicle (UAV) with a radio receiver around the area where they were last seen, and use the received wireless signal strength to fit a machine learning model that will estimate the hikers' position. Then, you'll relay this information to rescuers, who will try to reach that position by land. (Unfortunately, due to dense tree cover, the UAV will not be able to visually confirm their position.)

There is a complication, though - the UAV has a limited battery life, and therefore, limited flight time. You'll have to get an accurate estimate of the hikers' position in a very short time!



---

#### Objectives

In this experiment, you will:

* observe how the RBF kernel is used in a Gaussian Process Regression
* observe how the Gaussian Process Regression approximates the true function
* observe how Bayesian Optimization is used to decide which data point to acquire next.


---

#### Prerequisites

To complete this assignment, you should already have an account on AERPAW with the experimenter role, be part of a project, have all the necessary software to work with AERPAW experiments. You should also have already created an experiment with one UAV and one UGV. (See: [Hello, AERPAW](https://teaching-on-testbeds.github.io/hello-aerpaw/))

---

#### Citations

This experiment uses the Bayesian Optimization implementation of

>  Fernando Nogueira, "Bayesian Optimization: Open source constrained global optimization tool for Python," 2014. Available: https://github.com/fmfn/BayesianOptimization

It uses the AERPAW testbed:

> V. Marojevic, I. Guvenc, R. Dutta, M. Sichitiu, and B. Floyd, "Advanced Wireless for Unmanned Aerial Systems:5G Standardization, Research Challenges, and AERPAW Experimentation Platform", IEEE Vehic. Technol. Mag., vol. 15, no. 2. pp. 22-30, June 2020. DOI: 10.1109/MVT.2020.2979494.



and it uses a rover search implementation developed for the "Find a Rover" challenge:

> Kudyba, Paul, Jaya Sravani Mandapaka, Weijie Wang, Logan McCorkendale, Zachary McCorkendale, Mathias Kidane, Haijian Sun et al. "A UAV-assisted wireless localization challenge on AERPAW." arXiv preprint arXiv:2407.12180 (2024). https://arxiv.org/abs/2407.12180

The figures in the background section are from:

> Wang, Jie. "An intuitive tutorial to Gaussian processes regression." Computing in Science & Engineering (2023). https://arxiv.org/abs/2009.10862

with contributions by Yufei Zhen.


In [ ]:
!pip install bayesian-optimization==2.0.0 numpy==1.26.4 scikit_learn==1.5.2

After running the cell above, use Runtime > Restart Session from the Colab menu. Then, continue with the next cell.

In [ ]:
import numpy as np
import pandas as pd
import pickle

from bayes_opt import BayesianOptimization, acquisition
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, RBF

import geopy.distance

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from ipywidgets import interact, fixed, widgets
from mpl_toolkits import mplot3d
from IPython.core.interactiveshell import InteractiveShell
plt.rcParams['axes.formatter.useoffset'] = False

import moviepy.editor


In [ ]:
!git clone https://github.com/teaching-on-testbeds/uav-wireless-localization uav_wireless_localization

## Framing the problem

We are going to estimate the hikers' position based on the premise that the received signal strength is highest when the UAV is at the same latitude and longitude as the hikers.

We will frame our machine learning problem as follows:

* features $X$: latitude, longitude
* target variable $y$: received signal strength

In other words, given a coordinate (latitude and longitude) we want to predict the received signal strength at that location.

However, we don't really care if our model is bad at predicting the signal strength in places where it is low! Our *true* goal is to predict where the target variable will be highest. We will decide how "good" our model is by computing the mean squared error of the position estimate: the distance between the true location of the hikers, and the coordinate that our model predicts has the highest received signal strength.

## Run a rover search experiment on AERPAW

This sequence assumed you have already

* [created an account on AERPAW and joined a project](https://teaching-on-testbeds.github.io/hello-aerpaw/index#create-an-account-on-aerpaw) (one-time step)
* [created an experiment with a UGV and UAV and initiated development mode](https://teaching-on-testbeds.github.io/hello-aerpaw/index#start-an-experiment) (one-time step until you retire the experiment)

Then, you should have already investigated the effect of 

* the `kappa` hyperparameter
* the use of a `WhiteKernel`
* and the effect of setting the `noise_level_bounds` of the `WhiteKernel` and `length_scale_bounds` of the `RBF` kernel

Finally, when you are ready to test your model in the "real" search environment, you need to [set up access to experiment resources](https://teaching-on-testbeds.github.io/hello-aerpaw/index#access-experiment-resources), including:

* connecting your computer to the AERPAW VPN,
* opening an SSH session to the experiment console,
* opening an SSH session to the UAV VM (node 1 in the experiment),
* opening an SSH session to the UGV VM (node 2 in the experiment).
* if you will use QGroundControl: connecting QGroundControl, and setting up the `AFAR Rover.kml` geofence,


You may review [Hello, AERPAW](https://teaching-on-testbeds.github.io/hello-aerpaw/) as a reference for those last steps.

### Set up experiment

Now, we will configure applications that will run in the experiment - the radio transmitter (on UGV) and radio receiver (on UAV), and the Bayes search on the UAV.

Inside the SSH session on the UAV (node 1 in the experiment), install the `bayesian-optimization` package, which we will use to implement a Bayes search:

```
python3 -m pip install --target=/root/Profiles/vehicle_control/RoverSearch bayesian-optimization==2.0.0 numpy==1.26.4 scikit_learn==1.5.2
```

Download the `rover-search.py` script:

```
wget https://raw.githubusercontent.com/teaching-on-testbeds/uav-wireless-localization/refs/heads/main/rover_search.py -O  /root/Profiles/vehicle_control/RoverSearch/rover_search.py
```

and the signal power plotting script:

```
wget https://raw.githubusercontent.com/teaching-on-testbeds/hello-aerpaw/refs/heads/main/resources/plot_signal_power.py -O  /root/plot_signal_power.py
```


Still in the SSH session on the UAV (node 1 in the experiment), set up the applications that will run during our experiment - a radio receiver and a vehicle control script that implements our search with Gaussian Process Regression and Bayesian Optimization:


```
cd /root/Profiles/ProfileScripts/Radio 
cp Samples/startGNURadio-ChannelSounder-RX.sh startRadio.sh 

cd /root/Profiles/ProfileScripts/Vehicle
cp Samples/startRoverSearch.sh startVehicle.sh

cd /root
```


We will also change one parameter of the radio receiver. Run:

```
sed -i 's/^SPS=.*/SPS=8/' "/root/Profiles/ProfileScripts/Radio/Helpers/startchannelsounderRXGRC.sh"
```


Then, open the experiment script for editing

```
cd /root
nano /root/startexperiment.sh
```

and at the bottom of this file, remove the `#` comment sign next to `./Radio/startRadio.sh` and `./Vehicle/startVehicle.sh`, so that the end of the file looks like this:

```
./Radio/startRadio.sh
#./Traffic/startTraffic.sh
./Vehicle/startVehicle.sh
```

Hit Ctrl+O and then hit Enter to save the file. Then use Ctrl+X to exit and return to the terminal.

Now we will set up the UGV.

Inside an SSH session on the UGV (node 2 in the experiment), set up the applications that will run during our experiment - a radio transmitter and a vehicle GPS position logger:

```
cd /root/Profiles/ProfileScripts/Radio 
cp Samples/startGNURadio-ChannelSounder-TX.sh startRadio.sh 

cd /root/Profiles/ProfileScripts/Vehicle
cp Samples/startGPSLogger.sh startVehicle.sh

cd /root
```

We will also change one parameter of the radio transmitter. Run:

```
sed -i 's/^SPS=.*/SPS=8/' "/root/Profiles/ProfileScripts/Radio/Helpers/startchannelsounderTXGRC.sh"
```

Then, open the experiment script for editing

```
cd /root
nano /root/startexperiment.sh
```

and at the bottom of this file, remove the `#` comment sign next to `./Radio/startRadio.sh` and `./Vehicle/startVehicle.sh`, so that the end of the file looks like this:

```
./Radio/startRadio.sh
#./Traffic/startTraffic.sh
./Vehicle/startVehicle.sh
```

Hit Ctrl+O and then hit Enter to save the file. Then use Ctrl+X to exit and return to the terminal.

### Setup steps in experiment console

> **Note**: a video of this section is included at the end of the section.

On the experiment console, run

```
./startOEOConsole.sh
```

and add a column showing the position of each vehicle; in the experiment console run

```
add vehicle/position
```

and you will see a `vehicle/position` column added to the end of the table.


Then, in this experiment console window, set the start position of the UGV (node 2):

```
2 start_location 35.729 -78.699
```

and restart the controller on the UGV, so that the change of start location will take effect:

```
2 restart_cvm
```


If you are also watching in QGroundControl: In QGroundControl, the connection to the UGV may be briefly lost. Then it will return, and the UGV will be at the desired start location.


Even if you are not watching in QGroundControl, you will see in the `vehicle/position` column in the experiment console that the UGV (node 2) is at the position we have set.

In [ ]:
moviepy.editor.ipython_display("uav_wireless_localization/video/aerpaw_exp_console_an.mp4", width=800)


### Run experiment

Now we are ready to run an experiment!

#### Reset

> **Note**: a video of this section is included at the end of the section.

Start from a "clean slate" - on the UAV VM (node 1) and the UGV VM (node 2), run

```
cd /root
./stopexperiment.sh
```

to stop any sessions that may be lingering from previous experiments.

You should also reset the virtual channel emulator in between runs - on *either* VM (node 1 or node 2) run

```
./reset.sh
```

In [ ]:
moviepy.editor.ipython_display("uav_wireless_localization/video/aerpaw_reset_experiment_an.mp4", width=800)


#### Start experiment

> **Note**: a video of this section is included at the end of the section.

On the UGV VM (node 2), run

```
cd /root
./startexperiment.sh
```


In the terminal in which you are connected to the experiment console (with a table showing the state of the two vehicles) run


```
2 arm
```

In this table, for vehicle 2, you should see a "vehicle" and "txGRC" entry in the "screens" column.



On the UAV VM (node 1), run

```
cd /root
./startexperiment.sh
```

and wait a few moments, until you see the new processes appear in the "screens" column of the experiment console.


Then check the log of the vehicle navigation process by running (on the UAV VM, node 1):

```
tail -f Results/$(ls -tr Results/ | grep vehicle_log | tail -n 1 )
```

You should see a message

```
Guided command attempted. Waiting for safety pilot to arm
```

When you see this message, you can use Ctrl+C to stop watching the vehicle log.

In the experiment console, run

```
1 arm
```

to arm this vehicle. It will take off, reach altitude 50, and begin to search for the UGV.

While the search is ongoing, monitor the received signal power by running (on the UAV VM, node 1):

```
python3 plot_signal_power.py
```

and confirm that you see a stream of radio measurements, and that the signal is stronger when the UAV is close to the UGV.


You can monitor the position of the UAV by watching the flight in QGroundControl, or you can watch the position in the experiment console.


The experiment will run for 5 minutes from the time that the UAV reaches altitude. Then, the UAV will return to its original position and land.



When you see that the "screens" column in the experiment console no longer includes a "vehicle" entry for the UAV (node 1), its "mode" is LAND, and its altitude is very close to zero, then you know that the experiment is complete. You must wait for the experiment to completely finish, because the data files are only written at the end of the experiment.

In [ ]:
moviepy.editor.ipython_display("uav_wireless_localization/video/aerpaw_start_experiment_an.mp4", width=800, maxduration=2*60)


### Transfer data from AERPAW to Colab

Once your experiment is complete, you can transfer a CSV file of the search progress and the final optimizer state from AERPAW to your own laptop. Then, you can upload these files to Colab.

On the UAV VM (node 1), run

```
echo /root/Results/$(ls -tr Results/ | grep ROVER_SEARCH | tail -n 1 )
```

to get the name of the CSV file.


On the UAV VM (node 1), run

```
echo /root/Results/$(ls -tr Results/ | grep opt_final | tail -n 1 )
```

to get the name of the "pickled" optimizer file.

Then, in a *local* terminal (**not** inside any SSH session), run

```
scp -i ~/.ssh/id_rsa_aerpaw root@192.168.X.1:/root/Results/FILENAME .
```

where

* in place of the address with the `X`, you use the address you identified in the manifest,
* in place of `/root/Results/FILENAME` you use the filenames you just identified.
* the `.` at the end specifies that the file should be copied to your current working directory in your local terminal, but you can change this to another destination if you prefer.

You may be prompted for the passphrase for your key, if you set a passphrase when generating the key.



You will transfer a `ROVER_SEARCH_DATA.csv` file and an `opt_final.pickle` file to your laptop. Then, use the file browser in Google Colab to upload both files to Colab.

## Analyze experiment results

Once you have uploaded `ROVER_SEARCH_DATA.csv` and `opt_final.pickle` to Colab, we can analyze the experiment results.

In [ ]:
true_lat = 35.729
true_lon = -78.699

In [ ]:
from uav_wireless_localization.uav_utils import *

### Fitted model

Let us visualize the fitted model! In the following cells, change the file name to reflect the "ROVER_SEARCH_DATA.scv" and "opt_final.pickle" files you just uploaded.

In [ ]:
df_results = pd.read_csv('ROVER_SEARCH_DATA.csv')
with open('opt_final.pickle', 'rb') as handle:
    optimizer = pickle.load(handle)

then plot with

In [ ]:
vis_optimizer(optimizer, true_lat, true_lon)

In [ ]:
def plot_3D_with_opt(elev, azim):
    plot_3D(optimizer, elev, azim)

interact(plot_3D_with_opt, elev=widgets.IntSlider(min=-90, max=90, step=10, value=20),
          azim=widgets.IntSlider(min=-90, max=90, step=10, value=-70));


and look at the estimation error vs time with:

In [ ]:
plot_position_error_over_time(df_results, true_lat, true_lon)

Also note the fitted kernel parameters of the final model -

In [ ]:
optimizer._gp.kernel_.get_params()

## Modify the rover search settings

You will run this experiment for a different starting position of the ground vehicle (i.e. the hikers). To generate the new start position, you will put your net ID in the following cell, then run it.

In [ ]:
netID = "ff524"
seed = hash(netID) % (2**32)
np.random.seed(seed)

true_lat = np.random.uniform(BOUND_SE['lat'], BOUND_NE['lat'])
true_lon = np.random.uniform(BOUND_SE['lon'], BOUND_SW['lon'])
print(true_lat, true_lon)

All of the results you submit should be for *your* position of the ground vehicle (derived from your net ID in the cell below). You will submit results from two experiments, detailed below.

### Experiment 1 - default search settings

Now, you will re-do the "Run a rover search experiment on AERPAW". You will:

* Repeat "Setup steps in experiment console", but use the latitude and longitude printed by the previous cell (after you replace my net ID with yours!)
* Repeat the "Run experiment" steps (including "Reset" and "Start experiment").
* After your experiment is complete, make sure you save the data (CSV file and "pickle" file). Then, you'll do the "Transfer data from AERPAW to Colab" step. In the cell below, you'll modify the file names to reflect what you have just uploaded.

Then, you will repeat the analysis for your new experiment (with the "hikers' position" at this new location).


In [ ]:
df_results = pd.read_csv('ROVER_SEARCH_DATA.csv')
with open('opt_final.pickle', 'rb') as handle:
    optimizer = pickle.load(handle)

In [ ]:
vis_optimizer(optimizer, true_lat, true_lon)

In [ ]:
def plot_3D_with_opt(elev, azim):
    plot_3D(optimizer, elev, azim)

interact(plot_3D_with_opt, elev=widgets.IntSlider(min=-90, max=90, step=10, value=20),
          azim=widgets.IntSlider(min=-90, max=90, step=10, value=-70));


In [ ]:
plot_position_error_over_time(df_results, true_lat, true_lon)

In [ ]:
optimizer._gp.kernel_.get_params()

### Experiment 2 - modified search settings



Finally, you will use everything you have learned from your off-AERPAW experiments to change the model and search settings. Currently, the optimizer is configured as:

```python
    utility = acquisition.UpperConfidenceBound()

    optimizer = BayesianOptimization(
      f=None,
      pbounds={'lat': (MIN_LAT, MAX_LAT), 'lon': (MIN_LON, MAX_LON)},
      verbose=0,
      random_state=0,
      allow_duplicate_points=True,
      acquisition_function = utility
    )
    # set the kernel
    kernel = RBF()
    optimizer._gp.set_params(kernel = kernel)
```

but, you know these are not the ideal settings for finding the lost hikers. You can modify this - specifically, you can:

* set the `kappa` argument of the utility function, 
* add a `WhiteKernel()`, 
* and/or set the bounds of the kernel hyperparameters.

(you don't *have* to do all of these, just do what you believe will be effective based on your previous experiments).


On the UAV VM (node 1), run

```
nano /root/Profiles/vehicle_control/RoverSearch/rover_search.py
```

scroll to the part where the model is defined, and edit it. Then use Ctrl+O and Enter to save the file, and Ctrl+X to quit.

Next, you will:

* Repeat the "Run experiment" steps (including "Reset" and "Start experiment").
* After your experiment is complete, make sure you save the data (CSV file and "pickle" file) from your previous experiment in another directory (so they will not be overwritten by data from your new experiment). Then, you'll do the "Transfer data from AERPAW to Colab" step. In the cell below, you'll modify the file names to reflect what you have just uploaded.

Then, you will repeat the analysis for your new experiment.


In [ ]:
df_results = pd.read_csv('ROVER_SEARCH_DATA.csv')
with open('opt_final.pickle', 'rb') as handle:
    optimizer = pickle.load(handle)

In [ ]:
vis_optimizer(optimizer, true_lat, true_lon)

In [ ]:
def plot_3D_with_opt(elev, azim):
    plot_3D(optimizer, elev, azim)

interact(plot_3D_with_opt, elev=widgets.IntSlider(min=-90, max=90, step=10, value=20),
          azim=widgets.IntSlider(min=-90, max=90, step=10, value=-70));


In [ ]:
plot_position_error_over_time(df_results, true_lat, true_lon)

In [ ]:
optimizer._gp.kernel_.get_params()

Comment on the results, specifically:

* what changes did you make do the default settings of the optimizer and model?
* how has the appearance of the fitted model changed from the previous experiment, and why?
* what change do you see in the fitted model kernel parameters? 

📝 Specific requirements:

-   For full credit, you should achieve 10m or less estimation error by the end of the five-minute flight.
-   and, your fitted model should not show signs of severe overfitting or under-modeling.